# Elevator Project
The purpose of this project is to prototype elevator behaviour in a prototype 
style script and translate from python to cpp. This notebook behaves more like a playground w.r.t implementation to provide more insight into my thinking. 
For actual project deliverables consider the cpp solution.

## Assumptions
Below are a list of assumptions for the elevator implementation:

- An elevator has 3 main controllable interaction patterns:
    - Buttons on each floor that the elevator has access to in a 
      building/structure
        - UP/DOWN
    - Buttons within the elevator 
        - Id buttons representing each floor (usually numbers)
        - Security/Fire safety system override (requires key)
    - PA for communication into/out of the elevator
- The existence of more than one elevator in a building does not necessarily 
  change the behavior of an elevator (although some elevators may be defined 
  to have inaccessible floors)
- To ensure that an elevator doesn't try to optimize mobility and miss floors 
  that have been keyed in the elevator will maintain a priority ordered 
  schedule based on first in first out, and also continue in a direction 
  that has regularly been prioritized until reaching the furthest requested 
  floor in a direction before changing directions. This process will repeat 
  indefinitely while in operation and under direct input command
- An elevator will not move if it has not received an input command
- An elevator will move towards the input command floors with FIFO in mind
- An elevator has a fixed capacity and although some optimizations could be 
  made there is a risk of not following FIFO in general 
- If an elevator has been manually stopped for security/fire it cannot be 
  restarted until that external service has restarted it (e.g. regardless of 
  floor position the elevator should cease movement--note that it is ideal 
  if it can stop at a floor and will be assumed as true for the purposes 
  of this exercise)
- An elevator on the top floor has no up button
- An elevator on the bottom floor has no down button
- The lowest floor (ground) is floor 0
- The highest floor is defined on elevator construction
- Floor travel time also includes floor settling time

## Not Implemented

- Restricted floors (may require at least the following attributes to be 
  defined, and known security rules preventing piggy-backing)
    - It is assumed if a restricted floors feature was added camera 
      footage would be available in the elevator to know if the elevator 
      is empty prior to granting access to a restricted floor (given badge/key)

  ``` 
    self.isEmpty = <bool>
    self.hasAccessToRestrictedFloors = <bool>
    # List of unordered floors that must also have a card/key entry within 
    # the elevator to proceed
    # Otherwise the input request is ignored
    self.restrictedAccessFloors = [<int>,..]
  ```
- The historical/floors visited ids are meant for tracking visibility (in 
  the context of this project) and would likely require batching/id resetting 
  after a fixed duration to prevent the id's from getting out of control
- It is possible for accuracy to use type double for FLOOR_TRAVEL_TIME in 
  the future
- Safety and Security events can be inherited from parent (e.g. Building) 
  and thus will not be handled; starting with
- Database for storing elevator activities might be useful in the event 
  that their is a power outage while the elevator is operational
- Keeping track of movement state could be useful in the future
- Not including methods to interact directly with elevator accessor like 

```
class ElevatorDirection(Enum):
    UP = "Up"
    DOWN = "Down" 

def request_elevator_travel_from_floor(
  self, direction: ElevatorDirection, floorRequestedFrom: int) -> bool:
        """
        Request issued from a floor to travel up or down. 
        Lowest floor does not have a down option and highest floor does not have an up option.
        
        Args:
            direction (ElevatorDirection): Requested direction of travel from a floor with 
                elevator access (Options: UP, DOWN).
            floorRequestedFrom (int): Floor that elevator service is requested from.

        Returns:
            bool: True if the request was successful, False otherwise.
        """

        # This should not happen
        if (floorRequestedFrom > self.number_of_floors):
            return False

        # This should not happen
        if not isinstance(direction, ElevatorDirection):
            return False

        self.floors_to_visit.append(
            {
                "id": uuid.uuid4(),
                "floor": floorRequestedFrom,
                "requestSuccessful": True,
                "timeOfRequest": datetime.now()
            }
        )
        
        return True
```

```
# Fire alarm activity setting; elevator stops at the nearest floor when True
# Elevator does not move when True
self._isFireAlarmActive = False
# Security alarm setting; elevator stops at the nearest floor when True 
# TODO: Add building parent class with alarm reason codes as the building 
# may have specific methods for handling various security events
self._isSecurityAlarmActive = False
```

## Details
This section contains information on interaction I/O for the project.

### Constants
| Field | Type | Value | Description |
| --- | --- | --- | --- |
| FLOOR_TRAVEL_TIME | int | 10 | Single floor travel time |
| TRAVEL_SETTLING_TIME | int | 0 | Stop settling time after travel |

### Inputs
- List of floors to visit (e.g. elevator start=12 floor=2,9,1,32)

### Outputs 
- total travel time
- floors visited in order (e.g. 560 12,2,9,1,32)

In [ ]:
from datetime import datetime, timedelta
from enum import Enum
import uuid
import logging

class Elevator:
    """
    The Elevator class simulates the movement mechanics of an elevator.

    Attributes:
        number_of_floors (int): Total number floors serviced by the elevator
        floors_to_visit (list): Ordered collection of floors requested to visit. 
        floors_visited (list): Ordered collection of floors that have been visited.

    Methods:
        request_elevator_travel_to_floor(floor_number):
            Request issued from elevator to travel to specific floor. 
        process_elevator_travel_to_floor_request():
            Request processed for elevator to travel to next floor (First In First Out). 
        process_multiple_elevator_floor_requests():
            Request processed for entire floors_to_visit list (First In First Out). 
    """
    def __init__(self, number_of_floors: int, starting_floor: int, logger: logging.Logger):
        self._logger = logger
        # Lowest defined floor <int>
        self._lowest_floor = 0
        # Travel duration between floors <int>
        self._floor_travel_time = 10
        # Starting floor <int>
        self._starting_floor = starting_floor
        # Last known floor that the elevator stopped at <int>
        self._last_known_floor = starting_floor
        # Number of floors the elevator moves between <int>
        self.number_of_floors = number_of_floors
        # Initialization of floors to visit
        # Example: {
        #     id: <uuid>, # id representing the order in which 
        #     floor: <int>,
        #     requestSuccessful: <bool>,
        #     timeOfRequest: <datetime>
        # }
        self.floors_to_visit = []
        # Initialization of ordered historical floors visited
        # Example: {
        #     id: <uuid>,
        #     floor: <int>,
        #     requestSuccessful: <bool>,
        #     visitSuccessful: <bool>,
        #     timeOfRequest: <datetime>,
        #     timeOfVisit: <datetime>,
        #     travelDuration: <int>
        # }
        self.floors_visited = []
    
    def request_elevator_travel_to_floor(self, floor_number: int) -> bool:
        """
        Request issued from elevator to travel to specific floor. 
        
        Args:
            floor_number (int): Floor that elevator service is requested to.

        Returns:
            bool: True if the request was successful, False otherwise.
        """

        # This should not happen
        if (floor_number > self.number_of_floors 
            or floor_number < self._lowest_floor):
            self._logger.error(f"{floor_number} is not a valid or accessible "
                               + "floor. Elevator access request not processed.")
            return False

        self.floors_to_visit.append(
            {
                "id": uuid.uuid4(),
                "floor": floor_number,
                "requestSuccessful": True,
                "timeOfRequest": datetime.now()   
            }
        )
        
        return True       

    def process_elevator_travel_to_floor_request(self) -> str:
        """
        Request processed for elevator to travel to next floor 
        (First In First Out). 

        Returns:
            uuid: The uuid of processed travel request if successful, 
            None otherwise.
        """
        if (len(self.floors_to_visit) == 0):
            self._logger.error("No floors requested to visit. Elevator "
                               + "movement request not processed.")
            return

        # Set time cost of travel 
        # TODO: Consider making this its own function in the future
        _travel_duration = abs(
            self.floors_to_visit[0]["floor"] - self._last_known_floor) \
            * self._floor_travel_time

        # Add visited floor to history
        self.floors_visited.append(
            {
                "id": self.floors_to_visit[0]["id"],
                "floor": self.floors_to_visit[0]["floor"],
                "requestSuccessful": self.floors_to_visit[0]["requestSuccessful"],
                "timeOfRequest": self.floors_to_visit[0]["timeOfRequest"],
                "visitSuccessful": True,
                "timeOfVisit": self.floors_to_visit[0]["timeOfRequest"] + 
                    timedelta(seconds=_travel_duration),
                "travelDuration": _travel_duration
            }
        )

        # Remove processed entry
        self.floors_to_visit.pop(0)
        # Set last known floor to current floor for future processing
        self._last_known_floor = self.floors_visited[-1]["floor"]
        
        return self.floors_visited[-1]["id"]

    def process_multiple_elevator_floor_requests(self) -> bool:
        """
        Request processed for entire floors_to_visit listlevator to travel to next floor (First In First Out). 

        Returns:
            bool: True if the request was processed successfully, False otherwise.
        """
        # This should not happen
        if (len(self.floors_to_visit) == 0):
            self._logger.error("No floors requested to visit. Elevator movement request not processed.")
            return False

        # Instantiate processed floor information for logging outputs
        _processed_visit_request_ids = []
        _processed_visit_floors = [self._starting_floor]
        _processed_visit_duration_total = 0
        
        # Process floors
        for floor in self.floors_to_visit:
            # Append processed floor id for internal logging outputs
            _processed_visit_request_ids.append(self.process_elevator_travel_to_floor_request())

        # This should not happen
        if (len(_processed_visit_request_ids) == 0):
            return False

        for _processedVisitId in _processed_visit_request_ids:
            _historicalVisit = next((item for item in self.floors_visited if item.get("id") == _processedVisitId), None)
            _processed_visit_floors.append(_historicalVisit["floor"])
            _processed_visit_duration_total += _historicalVisit["travelDuration"]

        self._logger.info(f"Travel Duration: {_processed_visit_duration_total}(s)\n Floors Visited: {_processed_visit_floors}")
        return True        

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    input_floors = [1, 10, 6, 15, 85]
    elevator_starting_floor = 12
    test_elevator = Elevator(100, elevator_starting_floor, logging.getLogger(__name__))

    for floor in input_floors:
        test_elevator.request_elevator_travel_to_floor(floor)

    test_elevator.process_multiple_elevator_floor_requests()

2025-07-17 09:01:49,791 - INFO - Travel Duration: 240(s)
 Floors Visited: [12, 1, 10, 6]


True